In [ ]:
import openai 
from tenacity import * 

import openai 
PATH_TO_KEY="/home/data1/pie_data_sept_2023/openai_key.txt"
with open(PATH_TO_KEY, 'r') as f:
    key = f.read().strip()
PATH_TO_ORG="/home/data1/pie_data_sept_2023/openai_org.txt"
with open(PATH_TO_ORG, 'r') as f:
    org = f.read().strip()
openai.organization = org
openai.api_key = key

In [ ]:
def print_model_list(): 
    response = openai.Model.list()

    # Extract and print the model names
    for model in response["data"]:
        print(model["id"])

In [ ]:
print_model_list()

In [ ]:
# model = "gpt-4o-2024-05-13"
model = "gpt-3.5-turbo-0125"

In [ ]:
# PATH_TO_OUTPUT = "/data1/shypula/Project_CodeNet/processed"
# if not os.path.exists(PATH_TO_OUTPUT):
#     os.makedirs(PATH_TO_OUTPUT)
# # save each to a jsonl file, records and lines = True 
# train_records_df = pd.DataFrame(train_records)
# test_records_df = pd.DataFrame(test_records)
# val_records_df = pd.DataFrame(val_records)

# train_records_df.to_json(os.path.join(PATH_TO_OUTPUT, "train_descriptions_and_testcases.jsonl"), lines=True, orient="records")
# test_records_df.to_json(os.path.join(PATH_TO_OUTPUT, "test_descriptions_and_testcases.jsonl"), lines=True, orient="records")
# val_records_df.to_json(os.path.join(PATH_TO_OUTPUT, "val_descriptions_and_testcases.jsonl"), lines=True, orient="records")

import pandas as pd 

train_records = pd.read_json("/home/data1/processed/train_descriptions_and_testcases.jsonl", lines=True)
test_records = pd.read_json("/home/data1/processed/test_descriptions_and_testcases.jsonl", lines=True)
val_records = pd.read_json("/home/data1/processed/val_descriptions_and_testcases.jsonl", lines=True)

all_records = pd.concat([train_records, test_records, val_records])
print(all_records.shape)

In [ ]:
# print(all_records["description_string"].iloc[0].replace("\n+", "\n"))
import re 
canonicalize_newlines = lambda x: re.sub(r"\n+", "\n", x)
all_records["description_string_canonical"] = all_records["description_string"].apply(canonicalize_newlines)
print(all_records["description_string_canonical"].iloc[0])

In [ ]:
import sys 

sys.path.append("/home/alex/Documents/PennPhD/program_cal_diverse") 

from utils import open_ended 

In [ ]:
results = open_ended.prompt_gpt_to_generate_input_description(
    all_records["description_string_canonical"].iloc[0],
    open_ended.example_problem, open_ended.example_input, model=open_ended.default_model, temperature=0.5, n=2, top_p=1.0)

In [ ]:
results

In [ ]:
results = open_ended.prompt_gpt_to_write_testcase_generator(
    all_records["description_string_canonical"].iloc[0],
    open_ended.example_problem, open_ended.example_tcgen, model=open_ended.default_model, temperature=0.5, n=2, top_p=1.0)


In [ ]:
results

In [ ]:
# re-import 
import importlib 
importlib.reload(open_ended)

In [ ]:
html_examle = open_ended.make_n_html_from_programs(
    all_records["description_string_canonical"].iloc[0],
    n=1, 
    temp=0.0,
    top_p=1.0,
    model=open_ended.default_model
)


In [ ]:
# from IPython.display import HTML
# HTML(html_examle[0][0])    

In [ ]:
import joblib 

In [ ]:
from joblib import Parallel, delayed
import tqdm
import contextlib
import os

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

def process_problems_parallel(problems_with_ids, n, temp, top_p, model, output_directory):
    with tqdm_joblib(tqdm.tqdm(total=len(problems_with_ids))) as progress_bar:
        results = Parallel(n_jobs=10)(delayed(open_ended.make_n_html_from_programs)(
            desc, n, temp, top_p, model, problem_no=problem_id
        ) for problem_id, desc in problems_with_ids)
        
        for (problem_id, _), (html_outputs, input_descriptions, tc_gens, extract_args) in zip(problems_with_ids, results):
            problem_output_dir = os.path.join(output_directory, f"problem_{problem_id}")
            save_outputs_to_files(problem_id, problem_output_dir, html_outputs, input_descriptions, tc_gens, extract_args)
            progress_bar.update(1)  # Ensure the progress bar updates correctly per problem processed

def save_outputs_to_files(problem_id, output_directory, html_outputs, input_descriptions, tc_gens, extract_args):
    os.makedirs(output_directory, exist_ok=True)
    
    for i in range(len(html_outputs)):
        generation = i + 1
        html_file = os.path.join(output_directory, f"html_{problem_id}_{generation}.html")
        input_desc_file = os.path.join(output_directory, f"input_desc_{problem_id}_{generation}.txt")
        tc_gen_file = os.path.join(output_directory, f"tc_gen_{problem_id}_{generation}.txt")
        extract_args_file = os.path.join(output_directory, f"extract_args_{problem_id}_{generation}.txt")
        
        with open(html_file, 'w') as file:
            file.write(html_outputs[i])
        with open(input_desc_file, 'w') as file:
            file.write(input_descriptions[i])
        with open(tc_gen_file, 'w') as file:
            file.write(tc_gens[i])
        with open(extract_args_file, 'w') as file:
            file.write(extract_args[i])
            


# Example usage
# problems_with_ids = [('1', 'Some description 1'), ('2', 'Some description 2')]
# process_problems_parallel(problems_with_ids, 5, 0.8, 0.9, 'your_model_instance', '/path/to/output')


In [ ]:
all_records.columns

In [ ]:
# re-import 
import os 
import importlib 
import joblib 
importlib.reload(open_ended)

n_examples = 75
n_records = all_records.sample(n_examples)
descriptions = n_records["description_string_canonical"].tolist()
problem_ids = n_records["codenet_problem_id"].tolist()
problems_with_ids = list(zip(problem_ids, descriptions))
output_dir = "/home/data1/processed/outputs_v6_4o"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# process_problems_parallel(problems_with_ids, 3, 0.5, 1.0, open_ended.default_model, output_dir)
process_problems_parallel(problems_with_ids, 3, 0.75, 1.0, "gpt-4o-2024-05-13", output_dir)



In [ ]:
open_ended_final_path = "/home/data1/processed/open_ended_final/"
# read in the annotated notes and copy from /processed/output_* to /processed/open_ended_final based on columns 
import pandas as pd 
import shutil 

annotated_notes = pd.read_csv("/home/data1/processed/open_ended_final/examples_for_open_ended_gen_program_cal_diverse.csv")
# Index(['problem_id', 'specification_no', 'tcgen_no', 'Extract No',
#        'Additional Notes', 'Subjective Score (1-10)', 'Use? ',
#        'Specification'],
#       dtype='object')
annotated_notes = annotated_notes[annotated_notes["Use? "] == True]

annotated_notes.sort_values(by="Subjective Score (1-10)", ascending=False, inplace=True)
# reset index
annotated_notes = annotated_notes.reset_index(drop=True)
# cast the spec_no, tcgen, extract cols to int
for col in ["specification_no", "tcgen_no", "Extract No"]:
    annotated_notes[col] = annotated_notes[col].astype(int)
annotated_notes



In [ ]:
from tqdm import tqdm
import os 
def clean_string(input_string):
    # Replace non-breaking space with a regular space (or remove it entirely, if you prefer)
    cleaned_string = input_string.replace(u"\xa0", "").strip()
    return cleaned_string
for i, row in tqdm(annotated_notes.iterrows(), total=annotated_notes.shape[0]):
    problem_id = clean_string(row["problem_id"])
    spec_no = row["specification_no"]
    tcgen_no = row["tcgen_no"]
    extract_args = row["Extract No"]
    orig_dir = f"/home/data1/processed/outputs_v6_4o/problem_{problem_id}"
    dest_dir = f"{open_ended_final_path}/problem_{problem_id}"
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    # copy only the files that match the spec_no, tcgen_no, and extract_args
    shutil.copy2(clean_string(f"{orig_dir}/input_desc_{problem_id}_{spec_no}.txt"), f"{dest_dir}/input_desc_orig.txt")
    if not os.path.exists(f"{dest_dir}/input_desc_final.txt"):
        shutil.copy2(clean_string(f"{orig_dir}/input_desc_{problem_id}_{spec_no}.txt"), f"{dest_dir}/input_desc_final.txt")
    # copy the tc_gen
    shutil.copy2(clean_string(f"{orig_dir}/tc_gen_{problem_id}_{tcgen_no}.txt"), f"{dest_dir}/tc_gen_orig.txt")
    if not os.path.exists(f"{dest_dir}/tc_gen_final.txt"):
        shutil.copy2(clean_string(f"{orig_dir}/tc_gen_{problem_id}_{tcgen_no}.txt"), f"{dest_dir}/tc_gen_final.txt")
    # copy the extract_args
    shutil.copy2(clean_string(f"{orig_dir}/extract_args_{problem_id}_{extract_args}.txt"), f"{dest_dir}/extract_args_orig.txt")
    if not os.path.exists(f"{dest_dir}/extract_args_final.txt"):
        shutil.copy2(clean_string(f"{orig_dir}/extract_args_{problem_id}_{extract_args}.txt"), f"{dest_dir}/extract_args_final.txt")
    
    # also copy to "final"
    
    try: 
        orig_desc = all_records[all_records["codenet_problem_id"] == problem_id]["description_string_canonical"].iloc[0]
        with open(f"{dest_dir}/description.txt", 'w') as file:
            file.write(orig_desc)
    except Exception as e:
        print(f"Error with problem_id {problem_id}: {e}")
        

    

    
    
    

In [ ]:
all_records[all_records["codenet_problem_id"] == "p00050"]["description_string_canonical"].iloc[0]

In [ ]:
all_records[all_records["codenet_problem_id"] == "p00229"]

In [1]:

finalzed = {'p00050': True,
 'p00177': True,
 'p00229': True,
 'p00361': True,
 'p01020': True,
 'p01257': True,
 'p01508': True,
 'p01545': True,
 'p01639': True,
 'p02051': True,
 'p02181': True,
 'p02280': True,
 'p02731': True,
 'p02890': True,
 'p03222': True,
 'p03260': True,
 'p03666': True,
 'p03728': True,
 'p03774': True,
 'p03843': True,
 'p03907': True}

In [4]:
import random 

def tcgen_00361(): 
    N = random.randint(1, 10**4)  # Number of cities
    # 90% chance it is below 100 
    if random.random() < 0.9:
        M = random.randint(0, 100)
    else: 
        M = random.randint(0, 10**5)  # Number of roads

    roads = set()
    while len(roads) < M:
        s = random.randint(0, N-1)
        t = random.randint(0, N-1)
        if s != t:
            roads.add((s, t))
    
    roads = list(roads)
    
    test_case = f"{N} {len(roads)}\n"
    for s, t in roads:
        test_case += f"{s} {t}\n"
    
    return test_case.strip()

def make_testcase_01639():
        # Generating n, k, and x0 based on given constraints
        n = random.randint(1, 10**8)  # 1 ≤ n ≤ 10^8
        k = random.randint(1, n)  # 1 ≤ k ≤ n
        x0 = random.randint(0, 9)  # 0 ≤ x0 ≤ 9
        
        return n, k, x0

def tcgen_01639(): 
    n, k, x0 = make_testcase_01639()
    return f"{n} {k} {x0}"



def generate_binary_tree(n):
    """ Generates a random binary tree with n nodes. """
    nodes = list(range(n))
    random.shuffle(nodes)
    
    tree = {i: (-1, -1) for i in nodes}
    
    for i in range(n):
        if 2 * i + 1 < n:
            tree[nodes[i]] = (nodes[2 * i + 1], tree[nodes[i]][1])
        if 2 * i + 2 < n:
            tree[nodes[i]] = (tree[nodes[i]][0], nodes[2 * i + 2])

    return tree

def tcgen_02280():
    n = random.randint(1, 25)
    tree = generate_binary_tree(n)
    
    result = []
    result.append(str(n))

    for node in sorted(tree.keys()):
        left, right = tree[node]
        result.append(f"{node} {left} {right}")
    
    output_str = f"{n}\n" + "\n".join(result)
    return output_str


tc2tcgen = {
    "p00361": tcgen_00361,
    "p01639": tcgen_01639, 
    "p02280": tcgen_02280
}



open_ended_final_path = "/home/data1/processed/open_ended_final/"
PATH_TO_TESTCASES = "/home/alex/Documents/PennPhD/learning2perf/data/codenet/merged_test_cases"

# read in the annotated notes and copy from /processed/output_* to /processed/open_ended_final based on columns 
import pandas as pd 
import shutil 
import os 
import re 
import glob

from tqdm import tqdm
dataset = []
for problem_id, use in tqdm(finalzed.items()):
    if use:
        record = {}
        ## get the open-ended final files
        directory_path = os.path.join(open_ended_final_path, f"problem_{problem_id}")
        input_desc_final = os.path.join(directory_path, "input_desc_final.txt")
        extract_args_final = os.path.join(directory_path, "extract_args_final.txt")
        tc_gen_final = os.path.join(directory_path, "tc_gen_final.txt")
        with open(input_desc_final, "r") as f:
            input_desc = f.read().strip()
        with open(extract_args_final, "r") as f:
            extract_args = f.read().strip()
        with open(tc_gen_final, "r") as f:
            tc_gen = f.read().strip()
        record["problem_id"] = problem_id
        record["description_string"] = input_desc
        record["extract_args_fun"] = extract_args
        record["tc_gen_fun"] = tc_gen
        # grab the testcases
        path_to_tcs = os.path.join(PATH_TO_TESTCASES, problem_id)
        input_files = glob.glob(os.path.join(path_to_tcs, "input.*.txt"))
        output_files = glob.glob(os.path.join(path_to_tcs, "output.*.txt"))
        num_input_files = len(input_files)
        tc2input = {}
        tc2output = {}
        if num_input_files > 10: 
            input_tc_nos = [re.search(r"input\.(\d+)\.txt", f).group(1) for f in input_files]
            for input_tc_no in input_tc_nos:
                input_tc_path = os.path.join(path_to_tcs, f"input.{input_tc_no}.txt")
                output_tc_path = os.path.join(path_to_tcs, f"output.{input_tc_no}.txt")
                assert os.path.exists(input_tc_path)
                assert os.path.exists(output_tc_path), f"Missing output file for {input_tc_path}"
                with open(input_tc_path, "r") as f:
                    input_tc = f.read().strip()
                with open(output_tc_path, "r") as f:
                    output_tc = f.read().strip()
                tc2input[input_tc_no] = input_tc
                tc2output[input_tc_no] = output_tc
        elif problem_id in tc2tcgen:
            print(f"Generating testcases for {problem_id}")
            tc_gen_fun = tc2tcgen[problem_id]
            for i in tqdm(range(100), desc="Generating testcases", mininterval=1, maxinterval=1):
                input_tc = tc_gen_fun()
                output_tc = "NA"
                tc2input[str(i)] = input_tc
                tc2output[str(i)] = output_tc
                if (i % 10) == 0:
                    print(f"Generated {i} testcases")
            num_input_files = 100
        else:
            raise ValueError(f"No testcases or tcgen function for {problem_id}")
        record["problem_id"] = problem_id
        record["input_testcases"] = tc2input
        record["output_testcases"] = tc2output
        record["num_input_files"] = num_input_files
        dataset.append(record)

dataset_df = pd.DataFrame(dataset)
dataset_df.to_json("/home/data1/processed/open_ended_final/dataset.jsonl", lines=True, orient="records")

        
        
        
        
        



  0%|          | 0/21 [00:00<?, ?it/s]

Generating testcases for p00361


Generated 0 testcases
Generated 10 testcases
Generated 20 testcases
Generated 30 testcases
Generated 40 testcases
Generated 50 testcases
Generated 60 testcases


 19%|█▉        | 4/21 [00:00<00:01, 10.41it/s]

Generated 70 testcases
Generated 80 testcases
Generated 90 testcases
Generating testcases for p01639


Generating testcases: 100%|██████████| 100/100 [00:00<00:00, 526922.61it/s]


Generated 0 testcases
Generated 10 testcases
Generated 20 testcases
Generated 30 testcases
Generated 40 testcases
Generated 50 testcases
Generated 60 testcases
Generated 70 testcases
Generated 80 testcases
Generated 90 testcases
Generating testcases for p02280


100%|██████████| 21/21 [00:00<00:00, 51.14it/s]

Generated 0 testcases
Generated 10 testcases
Generated 20 testcases
Generated 30 testcases
Generated 40 testcases
Generated 50 testcases
Generated 60 testcases
Generated 70 testcases
Generated 80 testcases
Generated 90 testcases


In [5]:
dataset_df = pd.read_json("/home/data1/processed/open_ended_final/dataset.jsonl", lines=True)

In [6]:
# summarize number of input files
dataset_df["num_input_files"].describe()

count     21.000000
mean      99.523810
std       14.881596
min       35.000000
25%      102.000000
50%      102.000000
75%      104.000000
max      107.000000
Name: num_input_files, dtype: float64

In [7]:
# dataset_df[["num_input_files"], ["problem_id"]].sort_values(by="num_input_files", ascending=False)
dataset_df.sort_values(by="num_input_files", ascending=False)


,problem_id,description_string,extract_args_fun,tc_gen_fun,input_testcases,output_testcases,num_input_files
14,p03222,### Input Description:\n\nAn integer \(H\) (1 ...,"def extract_arguments(fh):\n H, W, K = map(...","def tcgen():\n H = random.randint(1, 100) ...","{'12': '2 6 1', '104': '2 3 3', '52': '20 8 1'...","{'12': '89', '104': '1', '52': '370317936', '2...",107
16,p03666,### Input Description:\n\nAn integer N (3 ≤ N ...,def extract_arguments(fh):\n # Read the lin...,import random\n\ndef tcgen():\n # Generate ...,{'12': '1788 122437383 943846071 90629745 1485...,"{'12': 'YES', '104': 'YES', '52': 'NO', '28': ...",105
19,p03843,### Input Description:\n\n1. An integer \(N\) ...,Certainly! Let's first write the `extract_argu...,## Note! not fully human checked\nimport rando...,"{'12': '4 8 4 2 3 2 4 1100', '52': '4 1 3 4 6 ...","{'12': '1', '52': '3', '28': '1', '84': '1', '...",104
18,p03774,### Input Description:\n\nAn integer \(N\) (1 ...,"def extract_arguments(fh):\n N, M = map(int...",import random\n\ndef make_testcase():\n # C...,"{'12': '3 4 10 6 -10 -10 3 4 0 4 2 3 3 9 3 5',...","{'12': '4 1 4', '52': '5 4 2 1 1', '28': '1 2 ...",104
17,p03728,### Input Description:\n\nAn integer \( N \) (...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef generate_permutation(n):\...,"{'12': '5 5 2 3 2 1', '52': '10 2 10 5 7 3 3 4...","{'12': '3', '52': '6', '28': '3', '84': '3', '...",104
15,p03260,### Input Description:\n\nAn integer A (1 ≤ A ...,"def extract_arguments(fh):\n A, B = map(int...",import random\n\ndef tcgen(): \n A = random...,"{'12': '-1 1', '52': '-2 -3', '28': '-2 -2', '...","{'12': 'Yes', '52': 'No', '28': 'No', '84': 'N...",104
13,p02890,### Canonicalized Input Description:\n\n1. An ...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef tcgen():\n # Generate ...,"{'12': '4 2 3 1 3', '52': '4 1 3 4 3', '28': '...","{'12': '4 2 1 0', '52': '4 2 1 0', '28': '4 1 ...",104
20,p03907,### Input Description:\n\nAn integer \(N\) (2 ...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef tcgen():\n # Step 1: G...,"{'12': '4 1 2 2 2 -1 1 -1 2', '52': '4 0 0 0 0...","{'12': '-1', '52': '-1', '28': '-1', '84': '-1...",103
12,p02731,### Input Description:\n\nAn integer \( L \) (...,def extract_arguments(fh):\n L = int(fh.rea...,import random\n\ndef tcgen():\n # Generate ...,"{'12': '-3', '52': '-22', '28': '19', '84': '1...","{'12': '-1.0', '52': '-394.37037037', '28': '2...",103
4,p01020,### Input Description:\n\n1. An integer \( N \...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef tcgen():\n # Set the n...,"{'12': '4 5 0 -2 8', '52': '5 2 -4 0 3 2', '28...","{'12': '6', '52': '-5', '28': '42', '84': '7',...",103


In [11]:
print(dataset_df[dataset_df["problem_id"] == "p01639"].iloc[0]["input_testcases"])
print(dataset_df[dataset_df["problem_id"] == "p01639"].iloc[0]["description_string"])


{'0': '42997399 20423435 2', '1': '84178519 35935250 8', '2': '38628289 20628831 3', '3': '82271175 75257574 1', '4': '66989838 44688341 9', '5': '3345116 294240 6', '6': '68668823 63213731 1', '7': '67204085 16358191 5', '8': '31987491 1335156 3', '9': '38340470 34999201 6', '10': '91607248 77211987 6', '11': '49129621 19679977 9', '12': '70206569 63643164 7', '13': '5996443 2098766 3', '14': '76222240 35707481 1', '15': '73297984 44350395 6', '16': '69054252 29015835 8', '17': '18016899 15447602 2', '18': '32350378 28979102 5', '19': '9156342 5168910 1', '20': '23507829 8614551 5', '21': '69023272 1275295 6', '22': '77247199 18195238 9', '23': '22593652 20935110 8', '24': '46398014 34714122 5', '25': '87897621 22475514 4', '26': '64785804 4718550 6', '27': '89936350 68211718 9', '28': '39395656 10093020 3', '29': '48729733 22412284 2', '30': '57810927 38630098 9', '31': '58240552 53188619 0', '32': '68105606 13659505 8', '33': '62898731 21957897 6', '34': '64142012 9199435 6', '35': 